In [7]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-241796
Azure region: eastus2
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-241796


In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "ml-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
 
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--max_iter': choice(100, 500, 1000),
        '--C': uniform(1e-6, 1e6)
    }
) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) ### YOUR CODE HERE ###

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=aml_compute,
                      environment=sklearn_env) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4) ### YOUR CODE HERE ###

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)

In [11]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_c506b459-0d31-4e26-bc66-8a236e788440
Web View: https://ml.azure.com/runs/HD_c506b459-0d31-4e26-bc66-8a236e788440?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-241796/workspaces/quick-starts-ws-241796&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-09-25T15:51:07.652741][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-09-25T15:51:08.3336523Z][SCHEDULER][INFO]Scheduling job, id='HD_c506b459-0d31-4e26-bc66-8a236e788440_0' 
[2023-09-25T15:51:08.4636924Z][SCHEDULER][INFO]Scheduling job, id='HD_c506b459-0d31-4e26-bc66-8a236e788440_1' 
[2023-09-25T15:51:08.6885358Z][SCHEDULER][INFO]Scheduling job, id='HD_c506b459-0d31-4e26-bc66-8a236e788440_3' 
[2023-09-25T15:51:08.5844793Z][SCHEDULER][INFO]Scheduling job, id='HD_c506b459-0d31-4e26-bc66-8a236e788440_2' 
[2023-09-25T15:51:08.638513][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [ ]:
best_run_metrics = best_run.get_metrics()

In [ ]:
best_run.get_file_names()

In [ ]:
best_run.download_file("/outputs/model.joblib", "./outputs/hyperdrive.joblib")

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_link = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_link)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='Accuracy',
    x=x,
    y=y,
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config)

In [ ]:
automl_run.wait_for_completion(show_output = True)

In [ ]:
assert(automl_run.get_status() == "Completed")

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, fitted_automl_model = automl_run.get_output()

In [ ]:
best_automl_run

In [ ]:
fitted_automl_model

In [ ]:
joblib.dump(fitted_automl_model, 'outputs/automl.joblib')

In [ ]:
aml_compute.delete()